# Promedio Diario $PM_{10}$

Para nuestra transformación de datos, utilizaremos las siguientes librerias:



In [1]:
import pandas as pd
import numpy as np

- Cargando dataset a un DataFrame de pandas

- Posteriormente normalizando la columna `Fecha` a tipo datetime

In [2]:
#Cargando archivos
df = pd.read_csv('../Datasets/PM10/PM10_depurado.csv')

# Normalizando fechas
df['FECHA'] = pd.to_datetime(df['FECHA'], unit='ns')
df.head()

,Unnamed: 0,FECHA,HORA,ACO,AJM,ATI,BJU,CAM,CUA,CUT,...,IZT,MER,PED,SAG,SFE,TAH,TLA,UIZ,VIF,XAL
0,0,2019-01-01,1,201.0,39.0,105.0,84.0,120.0,113.0,149.0,...,132.0,113.0,NaN,149.0,50.0,191.0,78.0,129.0,280.0,155.0
1,1,2019-01-01,2,218.0,27.0,128.0,115.0,135.0,158.0,211.0,...,202.0,141.0,NaN,149.0,54.0,225.0,73.0,125.0,257.0,236.0
2,2,2019-01-01,3,240.0,20.0,93.0,134.0,177.0,192.0,214.0,...,198.0,118.0,NaN,152.0,65.0,185.0,80.0,132.0,509.0,337.0
3,3,2019-01-01,4,231.0,10.0,86.0,124.0,204.0,143.0,178.0,...,192.0,154.0,NaN,217.0,33.0,172.0,84.0,136.0,611.0,271.0
4,4,2019-01-01,5,216.0,7.0,68.0,143.0,162.0,123.0,234.0,...,178.0,227.0,NaN,242.0,29.0,83.0,63.0,191.0,653.0,226.0


- Reacomodo del DataFrame , donde ahora tendremos una columna para las estaciones y sus correspondientes datos de medición
- Eliminando los datos `NaN`
- Comprobando que no queden datos `NaN` en el DataFrame

In [3]:
# Extraer nombre de columnas
cols = df.columns.tolist()

# Reacomodando dataframe
df = pd.melt(df, id_vars=['FECHA', 'HORA'], value_vars=cols[3:], var_name='station', value_name='measurement')

# df = df[~df['measurement'].isna()] otra forma de eliminar nan's
df = df.dropna(axis=0, how='any').reset_index(drop=True)

print("No. de columnas que contienen valores nulos ")
print(len(df.columns[df.isna().any()]))

print("No. de columnas que no contienen valores nulos")
print(len(df.columns[df.notna().all()]))

print("No total de columnas en el dataframe")
print(len(df.columns))
df.head()

No. de columnas que contienen valores nulos 
0
No. de columnas que no contienen valores nulos
4
No total de columnas en el dataframe
4


,FECHA,HORA,station,measurement
0,2019-01-01,1,ACO,201.0
1,2019-01-01,2,ACO,218.0
2,2019-01-01,3,ACO,240.0
3,2019-01-01,4,ACO,231.0
4,2019-01-01,5,ACO,216.0


- Ordenando el dataframe segun la fecha y la estación

In [4]:
# ordenando el dataframe por fecha y nombre de estacion
df = df.sort_values(['FECHA', 'station']).reset_index(drop=True)
df.head()

,FECHA,HORA,station,measurement
0,2019-01-01,1,ACO,201.0
1,2019-01-01,2,ACO,218.0
2,2019-01-01,3,ACO,240.0
3,2019-01-01,4,ACO,231.0
4,2019-01-01,5,ACO,216.0


### Calculando el promedio diario
Para el cálculo del promedio diario de $PM_{10}$ se utilizó la metodología descrita en la `NOM-025-SSA1-2014`, siguiendo la siguiente ecuación:

$$\bar{x}= \frac{1}{n} \displaystyle\sum_{i=1}^n x_i$$
$\bar{x}:$ promedio de 24 horas, 

$n:$ número de concentraciones horarias válidas

$x_i:$ concentraciones horarias válidas

- Camo datos extra obteniendo los valores minimos y maximos
- Tambien contabilizando cuantas horas fueron utilizadas para dicho promedio, columna(`N`)

In [5]:
# Calculando promedio diario, contando numero de horas usadas, obteniendo el maximo y minimo
df_PM1024h = df.groupby(['FECHA', 'station'], as_index=False).agg(
            PromDiario=('measurement', lambda x: round(x.mean(),0)), 
            N=('HORA', 'count'),
            Max = ('measurement', 'max'),
            Min = ('measurement', 'min'))
df_PM1024h.head()

,FECHA,station,PromDiario,N,Max,Min
0,2019-01-01,ACO,106.0,24,240.0,39.0
1,2019-01-01,AJM,36.0,24,91.0,7.0
2,2019-01-01,ATI,70.0,22,128.0,38.0
3,2019-01-01,BJU,84.0,24,143.0,21.0
4,2019-01-01,CAM,91.0,23,204.0,35.0


### Calculando Índice de calidad del aire

Donde:
$$\text{Índice}= (k \times (C \times BP_{Lo}))+I_{Lo}$$
$$k= \frac{I_{Hi}-I_{Lo}}{BP_{Hi}-BP_{Lo}}$$
 
$\text{Índice}:$ Valor del índice para el contaminante deseado.<br>
$C:$ valor redondeado para la concentración del contaminante.<br>
$k:$ constante de proporcionalidad estimada.<br>
$BP_{Hi}:$ valor del punto de corte que es mayor o igual a la concentración a evaluar.<br>
$BP_{Lo}:$ valor del punto de corte que es menor o igual a la concentración a evaluar.<br>
$I_{Hi}:$ valor del índice que corresponde al valor de $BP_{Hi}$<br>
$I_{Lo}:$ valor del índice que corresponde al valor de $BP_{Lo}$<br>


In [6]:
def AQI(df):
    CP = df['PromDiario']
    i = 0

    if 0 <= CP <= 40:
        k = (50-0)/(40-0)
        i = k * (CP - 0) + 0
    if 41 <= CP <= 75:
        k = (100-51)/(75-41)
        i = k * (CP - 41) + 51
    if 76 <= CP <= 214:
        k = (150-101)/(214-76)
        i = k * (CP - 76) + 101
    if 215 <= CP <= 354:
        k = (200-151)/(354-215)
        i = k * (CP - 215) + 151
    if 355 <= CP <= 424:
        k = (300-201)/(242-355)
        i = k * (CP - 355) + 201
    if 425 <= CP <= 504:
        k = (400-301)/(504-425)
        i = k * (CP - 425) + 301
    if 505 <= CP <= 604:
        k = (500-401)/(604-505)
        i = k * (CP - 505) + 401
    
    return round(i, 3)

In [7]:
df_PM1024h['indice'] = df_PM1024h.apply(AQI, axis=1)
df_PM1024h.head()

,FECHA,station,PromDiario,N,Max,Min,indice
0,2019-01-01,ACO,106.0,24,240.0,39.0,111.652
1,2019-01-01,AJM,36.0,24,91.0,7.0,45.000
2,2019-01-01,ATI,70.0,22,128.0,38.0,92.794
3,2019-01-01,BJU,84.0,24,143.0,21.0,103.841
4,2019-01-01,CAM,91.0,23,204.0,35.0,106.326


- Dejando los datos que utilicen mas de 17 horas para calcular el promedio diario
- Eliminando la columna `N`

In [8]:
df_PM1024h = df_PM1024h.query('N > 17')
df_PM1024h = df_PM1024h.drop(columns=['N'])
df_PM1024h.head()

,FECHA,station,PromDiario,Max,Min,indice
0,2019-01-01,ACO,106.0,240.0,39.0,111.652
1,2019-01-01,AJM,36.0,91.0,7.0,45.000
2,2019-01-01,ATI,70.0,128.0,38.0,92.794
3,2019-01-01,BJU,84.0,143.0,21.0,103.841
4,2019-01-01,CAM,91.0,204.0,35.0,106.326


- Cargando dataset de las estaciones con su correspondiente zona

In [9]:
# Cargando dataset de zonas
df_zonas = pd.read_csv('../Datasets/cat_estacion_depurado.csv')

# Eliminando columna 'Unnamed: 0'
df_zonas = df_zonas.drop(columns=['Unnamed: 0'])
# Remplazando los - por NaN
df_zonas = df_zonas.replace('-', np.nan)
# Eliminando lo NaN's
df_zonas = df_zonas.dropna(axis=0, how='any').reset_index(drop=True)
# Renombrando la columna
df_zonas = df_zonas.rename(columns={'cve_estac': 'station'})
df_zonas.head()

,station,nom_estac,Zona
0,ACO,Acolman,NE
1,AJU,Ajusco,SO
2,AJM,Ajusco Medio,SO
3,ATI,Atizapan,NO
4,BJU,Benito Ju�rez,CE


- Uniendo los datos con las zonas a cuales les corresponde

In [10]:
df_PM1024h = df_PM1024h.merge(df_zonas[['station', 'Zona']], on='station')
df_PM1024h.head()

,FECHA,station,PromDiario,Max,Min,indice,Zona
0,2019-01-01,ACO,106.0,240.0,39.0,111.652,NE
1,2019-01-02,ACO,73.0,158.0,41.0,97.118,NE
2,2019-01-03,ACO,60.0,88.0,24.0,78.382,NE
3,2019-01-04,ACO,70.0,110.0,42.0,92.794,NE
4,2019-01-05,ACO,30.0,59.0,17.0,37.500,NE


- Separando la fecha por, año, mes y dia



In [11]:
# Creando dataframe temporal para separar la fecha en años, meses y dias
dfdate = pd.DataFrame(columns = ['Year', 'Month', 'Day'])
# Extrayendo años
dfdate['Year'] = df_PM1024h['FECHA'].dt.year
# Extrayendo meses
dfdate['Month'] = df_PM1024h['FECHA'].dt.month
# Extrayendo dias
dfdate['Day'] = df_PM1024h['FECHA'].dt.day

# Elimiando columna de fecha en el dataframe original
df_PM1024h = df_PM1024h.drop(columns=['FECHA'])

# concatenacion del dataframe temporal que contiene la fecha separadas
data_PM1024h = pd.concat([dfdate.reset_index(drop=True), df_PM1024h.reset_index(drop=True)], axis=1)
data_PM1024h.head()

,Year,Month,Day,station,PromDiario,Max,Min,indice,Zona
0,2019,1,1,ACO,106.0,240.0,39.0,111.652,NE
1,2019,1,2,ACO,73.0,158.0,41.0,97.118,NE
2,2019,1,3,ACO,60.0,88.0,24.0,78.382,NE
3,2019,1,4,ACO,70.0,110.0,42.0,92.794,NE
4,2019,1,5,ACO,30.0,59.0,17.0,37.500,NE


- Guardando dataframe en un csv

In [12]:
data_PM1024h.to_csv('../Datasets/PM10/PDEstacion_PM10.csv', index=False)

### Calculo de promedio mensual

In [13]:
zona_ave = data_PM1024h.groupby(['Year','Month','Zona'], as_index=False).agg(
                                                                PromMensual=('PromDiario', lambda x: round(x.mean(),1)))
zona_ave.head()

,Year,Month,Zona,PromMensual
0,2019,1,CE,44.3
1,2019,1,NE,63.8
2,2019,1,NO,48.8
3,2019,1,SE,47.4
4,2019,1,SO,30.0


- Reordenando el dataframe en donde cada zona tenga una columa con sus valores correspondientes

In [14]:

zona_ave = pd.pivot_table(zona_ave, index=[zona_ave.Year, zona_ave.Month], columns='Zona', values='PromMensual').reset_index()

zona_ave.Year = zona_ave.Year.astype(int)

zona_ave.head()

Zona,Year,Month,CE,NE,NO,SE,SO
0,2019,1,44.3,63.8,48.8,47.4,30.0
1,2019,2,40.8,57.4,46.3,45.4,27.8
2,2019,3,44.6,58.3,51.1,49.1,35.3
3,2019,4,52.7,65.0,59.5,61.6,45.2
4,2019,5,51.3,64.2,58.0,58.7,44.0


- Guardando resultados en un csv

In [15]:
zona_ave.to_csv('../Datasets/PM10/PMZona_PM10.csv', index=False)